In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib
import numpy as np

FILE_PATH = 'data_after_filter.csv'
MODEL_FILE_NAME = 'model_xgboost.pkl'
RESULTS_FILE_NAME = 'raport_xgboost.txt'

try:
    df = pd.read_csv(FILE_PATH)
except FileNotFoundError:
    print(f"BŁĄD: Nie znaleziono pliku: {FILE_PATH}")
    exit()

df = df.loc[:, ~df.columns.duplicated()]

X = df.drop(['Resume_ID', 'Recruiter Decision'], axis=1, errors='ignore')
y = df['Recruiter Decision']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y 
)

print("Rozpoczynam trening XGBoost...")
ratio = float(np.sum(y == 0)) / np.sum(y == 1)

model = XGBClassifier(
    n_estimators=100,       # Liczba drzew
    learning_rate=0.1,      # Szybkość uczenia (mniejsza = dokładniej, ale wolniej)
    max_depth=3,            # Głębokość drzewa (zapobiega przeuczeniu)
    scale_pos_weight=ratio, # Balansowanie klas (kluczowe w rekrutacji!)
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

model.fit(X_train, y_train)
print("--- 5. Model XGBoost został wytrenowany ---")

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=['Odrzucony (0)', 'Zaproszony (1)'])

print(f"Dokładność: {accuracy:.4f}")
print("Raport klasyfikacji:\n", report)


train_accuracy = model.score(X_train, y_train)
test_accuracy = model.score(X_test, y_test)

print(f"Dokładność na zbiorze TRENINGOWYM: {train_accuracy:.4f}")
print(f"Dokładność na zbiorze TESTOWYM:    {test_accuracy:.4f}")

# --- 7. Ważność Cech ---
importances = model.feature_importances_
feature_importance_df = pd.DataFrame(
    {'Cecha': X.columns, 'Ważność': importances}
).sort_values(by='Ważność', ascending=False)

print("\nTop 10 najważniejszych cech wg XGBoost:")
print(feature_importance_df.head(10))

print(f"Wyniki poszczególnych prób CV: {cv_scores}")
print(f"Średni wynik CV: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# --- 8. Zapis wyników ---
with open(RESULTS_FILE_NAME, 'w', encoding='utf-8') as f:
    f.write(f"Model: XGBoost\n")
    f.write(f"Dokładność: {accuracy:.4f}\n\n")
    f.write("Raport:\n")
    f.write(report)
    f.write("\n\nWażność cech:\n")
    f.write(feature_importance_df.head(10).to_string())

joblib.dump(model, MODEL_FILE_NAME)
print(f"\nZapisano model do {MODEL_FILE_NAME}")

Rozpoczynam trening XGBoost...
--- 5. Model XGBoost został wytrenowany ---
Dokładność: 0.9567
Raport klasyfikacji:
                 precision    recall  f1-score   support

 Odrzucony (0)       0.99      0.95      0.97       244
Zaproszony (1)       0.83      0.96      0.89        56

      accuracy                           0.96       300
     macro avg       0.91      0.96      0.93       300
  weighted avg       0.96      0.96      0.96       300

Dokładność na zbiorze TRENINGOWYM: 0.9829
Dokładność na zbiorze TESTOWYM:    0.9567

Top 10 najważniejszych cech wg XGBoost:
                     Cecha   Ważność
0       Experience (Years)  0.397817
5           Projects Count  0.178343
3                 Job Role  0.032607
29                  Python  0.030143
22           Deep Learning  0.028299
8            Deep Learning  0.027577
15                  Python  0.024883
4   Salary Expectation ($)  0.023952
30                 Pytorch  0.023309
19              TensorFlow  0.021223

Uruchamiam 5

/Users/sebastianstarzec/miniconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:36:34] WARNING: /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [1]:
import pandas as pd
import numpy as np
import joblib
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score

# --- KONFIGURACJA ---
FILE_PATH = 'data_after_filter.csv'
MODEL_FILE_NAME = 'model_xgboost.pkl'
RESULTS_FILE_NAME = 'raport_xgboost.txt'

try:
    df = pd.read_csv(FILE_PATH)
except FileNotFoundError:
    print(f"BŁĄD: Nie znaleziono pliku: {FILE_PATH}")
    exit()

# Usuwanie duplikatów kolumn
df = df.loc[:, ~df.columns.duplicated()]

# Przygotowanie danych (X - cechy, y - cel)
X = df.drop(['Resume_ID', 'Recruiter Decision'], axis=1, errors='ignore')
y = df['Recruiter Decision']

# Podział na zbiór treningowy i testowy (70/30)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y 
)

print("Rozpoczynam proces trenowania i walidacji...")

# Obliczanie balansu klas dla parametru scale_pos_weight
ratio = float(np.sum(y == 0)) / np.sum(y == 1)

# --- PUNKT 5: TRENOWANIE I WALIDACJA ---

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for train_index, val_index in skf.split(X, y):
    # Podział na foldy
    X_train_cv, X_val_cv = X.iloc[train_index], X.iloc[val_index]
    y_train_cv, y_val_cv = y.iloc[train_index], y.iloc[val_index]
    
    # Model z ograniczoną głębokością (max_depth=3) przeciw przeuczeniu
    cv_model = XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        scale_pos_weight=ratio,
        eval_metric='logloss',
        random_state=42
    )
    
    cv_model.fit(X_train_cv, y_train_cv)
    score = cv_model.score(X_val_cv, y_val_cv)
    cv_scores.append(score)

cv_scores = np.array(cv_scores)
print(f"Wyniki poszczególnych prób CV: {cv_scores}")
print(f"Średni wynik CV: {cv_scores.mean():.4f}")
print(f"Stabilność (Odchylenie stand.): +/- {cv_scores.std():.4f}")

# 2. Trening modelu końcowego
model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    scale_pos_weight=ratio,
    eval_metric='logloss',
    random_state=42
)

model.fit(X_train, y_train)

# Ocena na zbiorze testowym
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=['Odrzucony (0)', 'Zaproszony (1)'])

print(f"Dokładność na zbiorze TESTOWYM: {accuracy:.4f}")
print("Raport klasyfikacji:\n", report)

# Porównanie wyników (diagnoza overfittingu)
train_acc = model.score(X_train, y_train)
print(f"Dokładność Treningowa: {train_acc:.4f}")
print(f"Dokładność Testowa:    {accuracy:.4f}")
print(f"Różnica (Gap):         {abs(train_acc - accuracy):.4f}")

# --- PUNKT 6: IMPLEMENTACJA (Ważność cech i zapis) ---

importances = model.feature_importances_
feature_importance_df = pd.DataFrame(
    {'Cecha': X.columns, 'Ważność': importances}
).sort_values(by='Ważność', ascending=False)

print("\nTop 10 najważniejszych cech wg XGBoost:")
print(feature_importance_df.head(10))

# Zapis wyników do pliku tekstowego
with open(RESULTS_FILE_NAME, 'w', encoding='utf-8') as f:
    f.write(f"Sredni wynik Cross-Validation: {cv_scores.mean():.4f}\n")
    f.write(f"Odchylenie standardowe CV: {cv_scores.std():.4f}\n")
    f.write(f"Dokladnosc na zbiorze testowym: {accuracy:.4f}\n\n")
    f.write("Raport klasyfikacji:\n")
    f.write(report)
    f.write("\n\nNajwazniejsze cechy (Top 10):\n")
    f.write(feature_importance_df.head(10).to_string())

# Zapis modelu do pliku .pkl
joblib.dump(model, MODEL_FILE_NAME)
print(f"\nModel zapisany pomyślnie jako: {MODEL_FILE_NAME}")

Rozpoczynam proces trenowania i walidacji...
Wyniki poszczególnych prób CV: [0.955 0.97  0.96  0.945 0.945]
Średni wynik CV: 0.9550
Stabilność (Odchylenie stand.): +/- 0.0095
Dokładność na zbiorze TESTOWYM: 0.9567
Raport klasyfikacji:
                 precision    recall  f1-score   support

 Odrzucony (0)       0.99      0.95      0.97       244
Zaproszony (1)       0.83      0.96      0.89        56

      accuracy                           0.96       300
     macro avg       0.91      0.96      0.93       300
  weighted avg       0.96      0.96      0.96       300

Dokładność Treningowa: 0.9829
Dokładność Testowa:    0.9567
Różnica (Gap):         0.0262

Top 10 najważniejszych cech wg XGBoost:
                     Cecha   Ważność
0       Experience (Years)  0.397817
5           Projects Count  0.178343
3                 Job Role  0.032607
29                  Python  0.030143
22           Deep Learning  0.028299
8            Deep Learning  0.027577
15                  Python  0.02488